In [1]:
import utils

f0_predictor = utils.get_f0_predictor('rmvpe',
                                        sampling_rate=44100,
                                        hop_length=512,
                                        device='cuda',
                                        threshold=0.05)

In [2]:
import librosa
wav, sr = librosa.load("notebook/test_audios/ref_audio/24k_xigua.wav", sr=44100)

In [3]:
import time


f0, uv = f0_predictor.compute_f0_uv(wav)


In [8]:
import yaml
config = yaml.safe_load(open('configs/pretrained/config.yaml'))

In [11]:
config.update()

In [2]:
all_data = []
with open("Data/backup/raw_text_en.list", 'r') as f:
    for line in f:
        all_data.append(line.strip())
with open("Data/raw_text.list", 'r') as f:
    for line in f:
        all_data.append(line.strip())

In [4]:
import random

random.shuffle(all_data)

valid = all_data[:10]
train = all_data[10:]

In [6]:
with open("Data/sovits_svc/train.list",'w') as f:
    for line in train:
        line = line.strip()
        f.write(line + '\n')

with open("Data/sovits_svc/dev.list",'w') as f:
    for line in valid:
        line = line.strip()
        f.write(line + '\n')

In [3]:
with open("Data/raw_text.list",'w') as f:
    for line in all_data:
        line = line.strip()
        f.write(line + '\n')

In [8]:
import os
from pathlib import Path

dir = "ft_ckpt/demo/audio_slice"
p = Path(dir)

In [2]:
p.glob("*.wav")

<generator object Path.glob at 0x7f5ac43d3c80>

In [9]:
def move_files_simple(source_dir, target_dir):
    source_path = Path(source_dir)
    target_path = Path(target_dir)
    
    # 确保目标目录存在
    target_path.mkdir(parents=True, exist_ok=True)
    
    # 移动所有文件
    for file_path in source_path.glob("*.wav"):
        if file_path.is_file():
            file_path.rename(target_path / file_path.name)
            print(f"Moved: {file_path.name}")

In [ ]:
move_files_simple(p, p.parent.parent/'sunyanzi')